# 01_Preprocess.ipynb
### Facial Emotion Recognition — Data Preprocessing
This notebook loads the FER2013 dataset (`fer2013.csv`), converts pixel strings to images, optionally applies face detection (MTCNN), and saves them in a structured `train/val` format.

In [4]:
# Install dependencies (run once)
!pip install mtcnn opencv-python pandas numpy tqdm pillow scikit-learn

In [7]:
import os
import numpy as np
import pandas as pd
from tqdm import tqdm
import cv2
from mtcnn.mtcnn import MTCNN
from sklearn.model_selection import train_test_split
from PIL import Image

DATA_PATH = '../data/fer2013.csv'
OUTPUT_DIR = '../data/cropped_faces'
TARGET_SIZE = 224
USE_MTCNN = True

In [8]:
os.makedirs(OUTPUT_DIR, exist_ok=True)
df = pd.read_csv(DATA_PATH)
df = df[df['pixels'].notnull()]
print('Dataset shape:', df.shape)
df.head()

Dataset shape: (35887, 3)


,emotion,pixels,Usage
0,0,70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...,Training
1,0,151 150 147 155 148 133 111 140 170 174 182 15...,Training
2,2,231 212 156 164 174 138 161 173 182 200 106 38...,Training
3,4,24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...,Training
4,6,4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...,Training


In [9]:
def pixels_to_image(pixels_str):
    arr = np.fromstring(pixels_str, dtype=int, sep=' ')
    img = arr.reshape(48, 48).astype('uint8')
    return cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)

detector = MTCNN() if USE_MTCNN else None

In [10]:
def detect_and_crop(img, target_size=224):
    if detector is None:
        return cv2.resize(img, (target_size, target_size))
    results = detector.detect_faces(img)
    if not results:
        return cv2.resize(img, (target_size, target_size))
    x, y, w, h = results[0]['box']
    x, y = max(0, x), max(0, y)
    face = img[y:y + h, x:x + w]
    return cv2.resize(face, (target_size, target_size))

In [11]:
X_paths, y_labels = [], []
temp_dir = os.path.join(OUTPUT_DIR, 'temp_raw')
os.makedirs(temp_dir, exist_ok=True)

for idx, row in tqdm(df.iterrows(), total=len(df)):
    img = pixels_to_image(row['pixels'])
    label = str(int(row['emotion']))
    out_path = os.path.join(temp_dir, f'{idx}_{label}.jpg')
    Image.fromarray(img).save(out_path)
    X_paths.append(out_path)
    y_labels.append(label)

100%|██████████| 35887/35887 [00:08<00:00, 4277.92it/s]


In [13]:
X_train, X_val, y_train, y_val = train_test_split(X_paths, y_labels, test_size=0.15, stratify=y_labels, random_state=42)
splits = [('train', X_train, y_train), ('val', X_val, y_val)]

In [ ]:
for split_name, X_split, y_split in splits:
    for img_path, label in tqdm(list(zip(X_split, y_split)), desc=f'Processing {split_name}'):
        img = cv2.imread(img_path)
        cropped = detect_and_crop(img, TARGET_SIZE)
        out_folder = os.path.join(OUTPUT_DIR, split_name, label)
        os.makedirs(out_folder, exist_ok=True)
        out_file = os.path.join(out_folder, os.path.basename(img_path))
        cv2.imwrite(out_file, cropped)

Processing train:   0%|          | 0/30503 [00:00<?, ?it/s]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 101ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step


Processing train:   0%|          | 1/30503 [00:00<3:14:32,  2.61it/s]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step


Processing train:   0%|          | 2/30503 [00:00<2:24:38,  3.51it/s]

...

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step


Processing train:  13%|█▎        | 3832/30503 [12:02<1:36:45,  4.59it/s]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 142ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step


Processing train:  13%|█▎        | 3833/30503 [12:02<2:03:17,  3.61it/s]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step


In [ ]:
print('Preprocessing completed successfully!')
print('Saved structure:')
!tree data/cropped_faces -L 2